## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,41.0,95,28,6.82,scattered clouds
1,1,Alice Springs,AU,-23.7000,133.8833,60.8,63,6,4.61,clear sky
2,2,Kapaa,US,22.0752,-159.3190,75.2,53,1,17.27,clear sky
3,3,Sakaiminato,JP,35.5500,133.2333,50.0,71,20,8.05,few clouds
4,4,Iracoubo,GF,5.4802,-53.2011,78.3,87,100,5.73,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Alice Springs,AU,-23.7000,133.8833,60.80,63,6,4.61,clear sky
2,2,Kapaa,US,22.0752,-159.3190,75.20,53,1,17.27,clear sky
4,4,Iracoubo,GF,5.4802,-53.2011,78.30,87,100,5.73,overcast clouds
5,5,Guerrero Negro,MX,27.9769,-114.0611,74.52,49,19,16.28,few clouds
7,7,Butaritari,KI,3.0707,172.7902,81.46,80,64,17.25,broken clouds
8,8,Walvis Bay,NaN,-22.9575,14.5053,77.00,38,0,8.05,clear sky
11,11,Barmer,IN,25.7500,71.3833,79.50,8,0,7.74,clear sky
12,12,El Bagre,CO,7.5927,-74.8086,86.50,65,94,1.10,overcast clouds
15,15,Carnarvon,AU,-24.8667,113.6333,75.20,78,40,14.97,scattered clouds
17,17,Rikitea,PF,-23.1203,-134.9692,76.30,61,43,1.32,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                419
City                   419
Country                415
Lat                    419
Lng                    419
Max Temp               419
Humidity               419
Cloudiness             419
Wind Speed             419
Current Description    419
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.count()

City_ID                415
City                   415
Country                415
Lat                    415
Lng                    415
Max Temp               415
Humidity               415
Cloudiness             415
Wind Speed             415
Current Description    415
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Alice Springs,AU,60.80,clear sky,-23.7000,133.8833,
2,Kapaa,US,75.20,clear sky,22.0752,-159.3190,
4,Iracoubo,GF,78.30,overcast clouds,5.4802,-53.2011,
5,Guerrero Negro,MX,74.52,few clouds,27.9769,-114.0611,
7,Butaritari,KI,81.46,broken clouds,3.0707,172.7902,
11,Barmer,IN,79.50,clear sky,25.7500,71.3833,
12,El Bagre,CO,86.50,overcast clouds,7.5927,-74.8086,
15,Carnarvon,AU,75.20,scattered clouds,-24.8667,113.6333,
17,Rikitea,PF,76.30,scattered clouds,-23.1203,-134.9692,
18,Saint Augustine,US,82.40,clear sky,29.8947,-81.3145,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(how="any").head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Alice Springs,AU,60.80,clear sky,-23.7000,133.8833,Desert Palms Alice Springs
2,Kapaa,US,75.20,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
4,Iracoubo,GF,78.30,overcast clouds,5.4802,-53.2011,Le Gite D Iracoubo
5,Guerrero Negro,MX,74.52,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
7,Butaritari,KI,81.46,broken clouds,3.0707,172.7902,Isles Sunset Lodge


In [17]:
hotel_df.count()

City                   415
Country                415
Max Temp               415
Current Description    415
Lat                    415
Lng                    415
Hotel Name             415
dtype: int64

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))